In [1]:
import os
import sys
import dimod
import matplotlib.pyplot as plt
import numpy as np

this_folder = os.path.dirname(os.path.abspath(os.getcwd()))
sys.path.insert(0, this_folder)

from utils import *
from optimized_qubos import *

dim = 2
initial_tensor = get_standard_tensor(dim) #% 2
suggested_optimal = 7 #Proved optimal for multiplying 2x2 matrices
origo = np.tensordot([0]*4, np.tensordot([0]*4, [0]*4, axes=0), axes=0)

In [2]:
strassen_tensors_np = [np.tensordot([0,0,0,1], np.tensordot([-1,0,1,0], [1,0,1,0], axes=0), axes=0),
           np.tensordot([1,1,0,0], np.tensordot([0,0,0,1], [-1,1,0,0], axes=0), axes=0),
           np.tensordot([-1,0,1,0], np.tensordot([1,1,0,0], [0,0,0,1], axes=0), axes=0),
           np.tensordot([1,0,0,1], np.tensordot([1,0,0,1], [1,0,0,1], axes=0), axes=0),
           np.tensordot([0,1,0,-1], np.tensordot([0,0,1,1], [1,0,0,0], axes=0), axes=0),
           np.tensordot([1,0,0,0], np.tensordot([0,1,0,-1], [0,1,0,1], axes=0), axes=0),
           np.tensordot([0,0,1,1], np.tensordot([1,0,0,0], [0,0,1,-1], axes=0), axes=0)]

In [3]:
def construct_all_tensors(sample, dim):
    positive_linear_vars = []
    tensors = []
    for i in range(suggested_optimal):
        x, y, z = [], [], []
        for j in range(dim**2):
            x.append(sample[str(i) + "x" + str(j)])
            y.append(sample[str(i) + "y" + str(j)])
            z.append(sample[str(i) + "z" + str(j)])
        tensors.append([x,y,z])
    return tensors

In [4]:
# Moving towards origo
bqm, poly = towards_user_defined_full(initial_tensor, dim, suggested_optimal)
print("Number of linear variables: ", len(bqm.linear))
print("Number of quadratic variables: ", len(bqm.quadratic))
sample, energy, sampleset = solve_bqm_in_leap(bqm, "Greedy")
tensors = construct_all_tensors(sample, dim)
for t in tensors:
    initial_tensor = initial_tensor - np.tensordot(t[0], np.tensordot(t[1], t[2], axes=0), axes=0)
if np.count_nonzero(initial_tensor.flatten()) == 0:
    print("Success!")
else:
    print("Fail!")
    print(tensors)

Number of linear variables:  667
Number of quadratic variables:  3093
Energy:  1.0
Fail!
[[[0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 1, 0]], [[0, 1, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0]], [[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1]], [[0, 1, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0]], [[0, 0, 0, 1], [0, 0, 1, 0], [0, 0, 1, 0]], [[0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1]], [[1, 0, 0, 0], [0, 1, 0, 0], [1, 1, 0, 0]]]


In [5]:
initial_tensor

array([[[ 1.,  0.,  0.,  0.],
        [-1.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.]]])

In [30]:
#print_solution(sample)

In [31]:
tensors

[[[0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 1, 0]],
 [[0, 1, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0]],
 [[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1]],
 [[0, 1, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0]],
 [[0, 0, 0, 1], [0, 0, 1, 0], [0, 0, 1, 0]],
 [[0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1]],
 [[1, 0, 0, 0], [0, 1, 0, 0], [1, 1, 0, 0]]]

In [28]:
strassen_tensors = [[[0,0,0,1], [-1,0,1,0], [1,0,1,0]],
                    [[1,1,0,0], [0,0,0,1], [-1,1,0,0]],
                    [[-1,0,1,0], [1,1,0,0], [0,0,0,1]],
                    [[1,0,0,1], [1,0,0,1], [1,0,0,1]],
                    [[0,1,0,-1], [0,0,1,1], [1,0,0,0]],
                    [[1,0,0,0], [0,1,0,-1], [0,1,0,1]],
                    [[0,0,1,1], [1,0,0,0], [0,0,1,-1]]]

mapping = {"x":0, "y":1, "z":2}
variables = dict()
for v in bqm.variables:
    if len(v) == 3:
        variables[v] = np.mod(strassen_tensors[int(v[0])][mapping[v[1]]][int(v[2])], 2)
    elif len(v) == 7:
        variables[v] = np.mod(strassen_tensors[int(v[0])][mapping[v[1]]][int(v[2])]*strassen_tensors[int(v[4])][mapping[v[5]]][int(v[6])], 2)
    elif len(v) == 11:
        variables[v] = np.mod(strassen_tensors[int(v[0])][mapping[v[1]]][int(v[2])]*strassen_tensors[int(v[4])][mapping[v[5]]][int(v[6])]*strassen_tensors[int(v[8])][mapping[v[9]]][int(v[10])], 2)
    elif len(v) == 15:
        variables[v] = np.mod(strassen_tensors[int(v[0])][mapping[v[1]]][int(v[2])]*strassen_tensors[int(v[4])][mapping[v[5]]][int(v[6])]*strassen_tensors[int(v[8])][mapping[v[9]]][int(v[10])]*strassen_tensors[int(v[12])][mapping[v[13]]][int(v[14])], 2)

#print(variables)
print(bqm.energy(variables))

48.0


In [29]:
#variables

In [9]:
bqm.energy(sample)

1.0

In [10]:
poly.energy(variables)

40.0

In [11]:
strassen_tensors = [[[0, 0, 0, 0], [1, 1, 1, 1], [0, 1, 0, 1]], [[1, 0, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0]], [[0, 1, 0, 1], [0, 0, 0, 1], [0, 1, 0, 1]], [[1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0]], [[0, 0, 0, 0], [1, 1, 0, 1], [0, 1, 1, 1]], [[0, 0, 0, 0], [0, 1, 0, 0], [1, 1, 1, 1]], [[0, 0, 0, 0], [0, 0, 1, 1], [1, 1, 0, 1]]]

mapping = {"x":0, "y":1, "z":2}
variables = dict()
for v in bqm.variables:
    if len(v) == 3:
        variables[v] = strassen_tensors[int(v[0])][mapping[v[1]]][int(v[2])] % 2
    elif len(v) == 7:
        variables[v] = ((strassen_tensors[int(v[0])][mapping[v[1]]][int(v[2])] % 2)*(strassen_tensors[int(v[4])][mapping[v[5]]][int(v[6])] % 2)) % 2
    elif len(v) == 11:
        variables[v] = ((strassen_tensors[int(v[0])][mapping[v[1]]][int(v[2])] % 2)*(strassen_tensors[int(v[4])][mapping[v[5]]][int(v[6])] % 2)*(strassen_tensors[int(v[8])][mapping[v[9]]][int(v[10])] % 2))%2
    elif len(v) == 15:
        variables[v] = (strassen_tensors[int(v[0])][mapping[v[1]]][int(v[2])] % 2)*(strassen_tensors[int(v[4])][mapping[v[5]]][int(v[6])] % 2)*(strassen_tensors[int(v[8])][mapping[v[9]]][int(v[10])] % 2)*(strassen_tensors[int(v[12])][mapping[v[13]]][int(v[14])] % 2)
        
print(poly.energy(variables))

-2.0


In [12]:
qubo = square_negative_sum(dict(), ["x", "y"], 1)

In [13]:
#sample, energy, sampleset = solve_bqm_in_leap(dimod.BinaryQuadraticModel(qubo), "Greedy")